In [1]:
import requests
from IPython.display import Markdown
from tqdm import tqdm, tqdm_notebook
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
import altair as alt
from requests.utils import quote
import os
from datetime import timedelta
from mod import alt_theme

In [2]:
fmt = "{:%Y-%m-%d}"

# Can optionally use number of days to choose dates
n_days = 60
end_date = fmt.format(pd.datetime.today())
start_date = fmt.format(pd.datetime.today() - timedelta(days=n_days))

renderer = "kaggle"
github_orgs = ["jupyterhub", "jupyter", "jupyterlab", "jupyter-widgets", "ipython", "binder-examples", "nteract"]
bot_names = ["stale", "codecov", "jupyterlab-dev-mode", "henchbot"]

In [3]:
# Parameters
renderer = "kaggle"
github_orgs = ["jupyterhub", "jupyter", "jupyterlab", "jupyter-widgets", "ipython", "binder-examples"]
start_date = "2019-07-01"
end_date = "2019-08-01"


In [4]:
alt.renderers.enable(renderer);
alt.themes.register('my_theme', alt_theme)
alt.themes.enable("my_theme")

ThemeRegistry.enable('my_theme')

In [5]:
# Discourse API key
api = {'Api-Key': os.environ['DISCOURSE_API_KEY'],
       'Api-Username': os.environ['DISCOURSE_API_USERNAME']}

In [6]:
# Discourse
def topics_to_markdown(topics, n_list=10):
    body = []
    for _, topic in topics.iterrows():
        title = topic['fancy_title']
        slug = topic['slug']
        posts_count = topic['posts_count']
        url = f'https://discourse.jupyter.org/t/{slug}'
        body.append(f'* [{title}]({url}) ({posts_count} posts)')
    body = body[:n_list]
    return '\n'.join(body)

def counts_from_activity(activity):
    counts = activity.groupby('category_id').count()['bookmarked'].reset_index()
    counts['parent_category'] = None
    for ii, irow in counts.iterrows():
        if parent_categories[irow['category_id']] is not None:
            counts.loc[ii, 'parent_category'] = parent_categories[irow['category_id']]

    counts['category_id'] = counts['category_id'].map(lambda a: category_mapping[a])
    counts['parent_category'] = counts['parent_category'].map(lambda a: category_mapping[a] if a is not None else 'parent')
    is_parent = counts['parent_category'] == 'parent'
    counts.loc[is_parent, 'parent_category'] = counts.loc[is_parent, 'category_id'] 
    counts['parent/category'] = counts.apply(lambda a: a['parent_category']+'/'+a['category_id'], axis=1)
    counts = counts.sort_values(['parent_category', 'bookmarked'], ascending=False)
    return counts

# Community forum activity

The [Jupyter Community Forum](https://discourse.jupyter.org) is a place for Jovyans across the
community to talk about Jupyter tools in interactive computing and how they fit into their
workflows. It's also a place for developers to share ideas, tools, tips, and help one another.

Below are a few updates from activity in the Discourse. For more detailed information about
the activity on the Community Forum, check out these links:

* [The users page](https://discourse.jupyter.org/u) has information about user activity
* [The top posts page](https://discourse.jupyter.org/top) contains a list of top posts, sorted
  by various metrics.

In [7]:
# Get categories for IDs
url = "https://discourse.jupyter.org/site.json"
resp = requests.get(url, headers=api)
category_mapping = {cat['id']: cat['name'] for cat in resp.json()['categories']}
parent_categories = {cat['id']: cat.get("parent_category_id", None) for cat in resp.json()['categories']}

In [8]:
# Base URL to use
url = "https://discourse.jupyter.org/latest.json"

## Topics with lots of likes

"Likes" are a way for community members to say thanks for a helpful post, show their
support for an idea, or generally to share a little positivity with somebody else.
These are topics that have generated lots of likes in recent history.

In [9]:
params = {"order": "likes", "ascending": "False"}
resp = requests.get(url, headers=api, params=params)

# Topics with the most likes in recent history
liked = pd.DataFrame(resp.json()['topic_list']['topics'])
Markdown(topics_to_markdown(liked))

* [Introduce yourself!](https://discourse.jupyter.org/t/introduce-yourself) (117 posts)
* [{WIP} Re-Design of Jupyter.org Website](https://discourse.jupyter.org/t/wip-re-design-of-jupyter-org-website) (68 posts)
* [JupyterHub, Binder workshop in Oslo, Sept. 5-8](https://discourse.jupyter.org/t/jupyterhub-binder-workshop-in-oslo-sept-5-8) (49 posts)
* [Microsoft Word Integration (Intern Project)](https://discourse.jupyter.org/t/microsoft-word-integration-intern-project) (25 posts)
* [Jupyter Community Calls](https://discourse.jupyter.org/t/jupyter-community-calls) (39 posts)
* [What&rsquo;s an alternative to the word &ldquo;Fellow&rdquo; or &ldquo;Fellowship&rdquo;?](https://discourse.jupyter.org/t/whats-an-alternative-to-the-word-fellow-or-fellowship) (16 posts)
* [Help us choose an updated &ldquo;Launch Binder&rdquo; badge!](https://discourse.jupyter.org/t/help-us-choose-an-updated-launch-binder-badge) (21 posts)
* [The Binder Federation](https://discourse.jupyter.org/t/the-binder-federation) (14 posts)
* [PyCon 2019 and mybinder.org](https://discourse.jupyter.org/t/pycon-2019-and-mybinder-org) (17 posts)
* [Governance Office Hours Meeting Minutes](https://discourse.jupyter.org/t/governance-office-hours-meeting-minutes) (25 posts)

## Active topics on the Community Forum

These are topics with lots of activity in recent history.

In [10]:
params = {"order": "posts", "ascending": "False"}
resp = requests.get(url, headers=api, params=params)

# Topics with the most posts in recent history
posts = pd.DataFrame(resp.json()['topic_list']['topics'])
Markdown(topics_to_markdown(posts))

* [Introduce yourself!](https://discourse.jupyter.org/t/introduce-yourself) (117 posts)
* [{WIP} Re-Design of Jupyter.org Website](https://discourse.jupyter.org/t/wip-re-design-of-jupyter-org-website) (68 posts)
* [Tip: embed custom github content in a Binder link with nbgitpuller](https://discourse.jupyter.org/t/tip-embed-custom-github-content-in-a-binder-link-with-nbgitpuller) (52 posts)
* [JupyterHub, Binder workshop in Oslo, Sept. 5-8](https://discourse.jupyter.org/t/jupyterhub-binder-workshop-in-oslo-sept-5-8) (49 posts)
* [Jupyter Community Calls](https://discourse.jupyter.org/t/jupyter-community-calls) (39 posts)
* [Would a &ldquo;The Littlest Binder&rdquo; be useful?](https://discourse.jupyter.org/t/would-a-the-littlest-binder-be-useful) (31 posts)
* [Plugins of considerations](https://discourse.jupyter.org/t/plugins-of-considerations) (28 posts)
* [Is there a free (even ad-supported) public JupyterHub available?](https://discourse.jupyter.org/t/is-there-a-free-even-ad-supported-public-jupyterhub-available) (26 posts)
* [A proposal for JupyterHub communications](https://discourse.jupyter.org/t/a-proposal-for-jupyterhub-communications) (26 posts)
* [Governance Office Hours Meeting Minutes](https://discourse.jupyter.org/t/governance-office-hours-meeting-minutes) (25 posts)

In [11]:
counts = counts_from_activity(posts)
alt.Chart(data=counts, width=700, height=300, title="Activity by category").mark_bar().encode(
    x=alt.X("parent/category", sort=alt.Sort(counts['category_id'].values.tolist())),
    y="bookmarked",
    color="parent_category"
)

Chart({
  data:                          category_id  bookmarked     parent_category  \
  14                     accessibility           1      Special Topics   
  12                    Admin requests           1       Site Feedback   
  10                               Q&A           1                 Q&A   
  15                         nbconvert           1            Notebook   
  1                               Meta           3                Meta   
  2                      announcements           1                Meta   
  9                         JupyterLab           2          JupyterLab   
  7                            discuss           3          JupyterHub   
  3                         JupyterHub           2          JupyterHub   
  0   Zero to JupyterHub on Kubernetes           1          JupyterHub   
  13                        Governance           1          Governance   
  11                Enterprise Gateway           2  Enterprise Gateway   
  4                     Community Chat           3      Community Chat   
  8                            discuss           4              Binder   
  5                             Binder           3              Binder   
  6                          BinderHub           1              Binder   
  
                                  parent/category  
  14                 Special Topics/accessibility  
  12                 Site Feedback/Admin requests  
  10                                      Q&A/Q&A  
  15                           Notebook/nbconvert  
  1                                     Meta/Meta  
  2                            Meta/announcements  
  9                         JupyterLab/JupyterLab  
  7                            JupyterHub/discuss  
  3                         JupyterHub/JupyterHub  
  0   JupyterHub/Zero to JupyterHub on Kubernetes  
  13                        Governance/Governance  
  11        Enterprise Gateway/Enterprise Gateway  
  4                 Community Chat/Community Chat  
  8                                Binder/discuss  
  5                                 Binder/Binder  
  6                              Binder/BinderHub  ,
  encoding: FacetedEncoding({
    color: Color({
      field: 'parent_category',
      type: 'nominal'
    }),
    x: X({
      field: 'parent/category',
      sort: Sort(['accessibility', 'Admin requests', 'Q&A', 'nbconvert', 'Meta', 'announcements', 'JupyterLab', 'discuss', 'JupyterHub', 'Zero to JupyterHub on Kubernetes', 'Governance', 'Enterprise Gateway', 'Community Chat', 'discuss', 'Binder', 'BinderHub']),
      type: 'nominal'
    }),
    y: Y({
      field: 'bookmarked',
      type: 'quantitative'
    })
  }),
  height: 300,
  mark: 'bar',
  title: 'Activity by category',
  width: 700
})

## Recently-created topics

These are topics that were recently created, sorted by the amount of activity
in each one.

In [12]:
params = {"order": "created", "ascending": "False"}
resp = requests.get(url, headers=api, params=params)

# Sort created by the most posted for recently-created posts
created = pd.DataFrame(resp.json()['topic_list']['topics'])
created = created.sort_values('posts_count', ascending=False)
Markdown(topics_to_markdown(created))

* [Governance Questions 1: Community](https://discourse.jupyter.org/t/governance-questions-1-community) (9 posts)
* [Benefits of the classic UI and use cases for classic over JupyterLab (Was: Why is Tim not moving to lab?)](https://discourse.jupyter.org/t/benefits-of-the-classic-ui-and-use-cases-for-classic-over-jupyterlab-was-why-is-tim-not-moving-to-lab) (7 posts)
* [Nbviewer not showing updated version of Jupyter Notebook](https://discourse.jupyter.org/t/nbviewer-not-showing-updated-version-of-jupyter-notebook) (4 posts)
* [Is there a way to save the output of a cell?](https://discourse.jupyter.org/t/is-there-a-way-to-save-the-output-of-a-cell) (3 posts)
* [Jupyter not starting in Win 10](https://discourse.jupyter.org/t/jupyter-not-starting-in-win-10) (3 posts)
* [Help! How do I deal with read-only notebooks?](https://discourse.jupyter.org/t/help-how-do-i-deal-with-read-only-notebooks) (3 posts)
* [Is it possible to launch a repository of Jupyter Notebooks, but have the link open in a specific notebook rather than the Home Directory?](https://discourse.jupyter.org/t/is-it-possible-to-launch-a-repository-of-jupyter-notebooks-but-have-the-link-open-in-a-specific-notebook-rather-than-the-home-directory) (3 posts)
* [Sudospawner and internal_ssl](https://discourse.jupyter.org/t/sudospawner-and-internal-ssl) (2 posts)
* [How to participate and what to expect in the governance refactor](https://discourse.jupyter.org/t/how-to-participate-and-what-to-expect-in-the-governance-refactor) (2 posts)
* [JupyterHub/Binder Community Call, November 2019](https://discourse.jupyter.org/t/jupyterhub-binder-community-call-november-2019) (2 posts)

In [13]:
counts = counts_from_activity(created)
alt.Chart(data=counts, width=700, height=300, title="Activity by category").mark_bar().encode(
    x=alt.X("parent/category", sort=alt.Sort(counts['category_id'].values.tolist())),
    y="bookmarked",
    color="parent_category"
)

Chart({
  data:                         category_id  bookmarked parent_category  \
  5                               Q&A           1             Q&A   
  8                          Notebook           3        Notebook   
  1                              jobs           1            Meta   
  4                        JupyterLab           2      JupyterLab   
  2                        JupyterHub           2      JupyterHub   
  0  Zero to JupyterHub on Kubernetes           1      JupyterHub   
  7                        Governance           4      Governance   
  3                    Community Chat           1  Community Chat   
  6                  mybinder.org ops           1          Binder   
  
                                 parent/category  
  5                                      Q&A/Q&A  
  8                            Notebook/Notebook  
  1                                    Meta/jobs  
  4                        JupyterLab/JupyterLab  
  2                        JupyterHub/JupyterHub  
  0  JupyterHub/Zero to JupyterHub on Kubernetes  
  7                        Governance/Governance  
  3                Community Chat/Community Chat  
  6                      Binder/mybinder.org ops  ,
  encoding: FacetedEncoding({
    color: Color({
      field: 'parent_category',
      type: 'nominal'
    }),
    x: X({
      field: 'parent/category',
      sort: Sort(['Q&A', 'Notebook', 'jobs', 'JupyterLab', 'JupyterHub', 'Zero to JupyterHub on Kubernetes', 'Governance', 'Community Chat', 'mybinder.org ops']),
      type: 'nominal'
    }),
    y: Y({
      field: 'bookmarked',
      type: 'quantitative'
    })
  }),
  height: 300,
  mark: 'bar',
  title: 'Activity by category',
  width: 700
})

## User activity in the Community Forum

**Top posters**

These people have posted lots of comments, replies, answers, etc in the community forum.

In [14]:
def plot_user_data(users, column, sort=False):
    plt_data = users.sort_values(column, ascending=False).head(50)
    x = alt.X("username", sort=plt_data['username'].tolist()) if sort is True else 'username'
    ch = alt.Chart(data=plt_data).mark_bar().encode(
        x=x,
        y=column
    )
    return ch

In [15]:
url = "https://discourse.jupyter.org/directory_items.json"
params = {"period": "quarterly", "order": "post_count"}
resp = requests.get(url, headers=api, params=params)

# Topics with the most likes in recent history
users = pd.DataFrame(resp.json()['directory_items'])
users['username'] = users['user'].map(lambda a: a['username'])

In [16]:
plot_user_data(users.head(50), 'post_count')

Chart({
  data:       id  likes_received  likes_given  topics_entered  topic_count  \
  0      6             101          163             257            7   
  1    220              20            2              72            0   
  2      3             103           86             102            9   
  3     73              14           31             288            3   
  4      8              36            5              56            0   
  5     11              16           78             261            0   
  6    116              22           19              89            1   
  7    198               8           18              85            1   
  8      7              26           33              69            0   
  9     37               6           16               7            1   
  10   303               4           10             119            3   
  11    99              16           18              73            4   
  12   217              10           17              34            1   
  14   268              11            4               5            0   
  13   121              14            3              11            1   
  15   747               3            0              19            4   
  16    59              13            9              36            0   
  17   674               6            5              61            4   
  18    30              15           61              37            3   
  19   485               2           28              35            0   
  20   229               5            5              84            0   
  21   234              25            4              17            2   
  24   679              12            3              23            0   
  26   590               1            1              20            2   
  25    72               8            4              45            0   
  23   884               2            0              41            7   
  22   737               2            2              43            2   
  27   771              13            3               4            0   
  28  1004               0            2               8            1   
  29   803               4            0              11            1   
  30   412               0            0               3            0   
  31   920               1            0              72            2   
  32    77               7            8             139            0   
  38  1005               8            3               9            3   
  42   660               1            8               4            0   
  41    24               7            5             148            2   
  40   222               9            2               3            0   
  39   655               4            0              15            1   
  37   904               0            5              15            5   
  36   831               8            3              14            2   
  35   570               5            0               3            0   
  34   731               1            0               6            1   
  33   834               0            0               9            1   
  43   761               0            0               5            1   
  44   874               0            0               4            1   
  45   926               4            2               8            2   
  46   886               0            0               9            1   
  47   164               3           10              37            0   
  48   959               0            0              13            0   
  49    34               6            2               8            0   
  
      post_count  posts_read  days_visited  \
  0          104         951            86   
  1           53         337            51   
  2           50         473            77   
  3           35        1194            70   
  4           25         263            89   
  5           25        1062       

**Forum users, sorted by likes given**

These are Community Forum members that "liked" other people's posts. We appreciate
anybody taking the time to tell someone else they like what they're shared!

In [17]:
plot_user_data(users.head(50), 'likes_given')

Chart({
  data:       id  likes_received  likes_given  topics_entered  topic_count  \
  0      6             101          163             257            7   
  2      3             103           86             102            9   
  5     11              16           78             261            0   
  18    30              15           61              37            3   
  8      7              26           33              69            0   
  3     73              14           31             288            3   
  19   485               2           28              35            0   
  6    116              22           19              89            1   
  11    99              16           18              73            4   
  7    198               8           18              85            1   
  12   217              10           17              34            1   
  9     37               6           16               7            1   
  10   303               4           10             119            3   
  47   164               3           10              37            0   
  16    59              13            9              36            0   
  32    77               7            8             139            0   
  42   660               1            8               4            0   
  41    24               7            5             148            2   
  17   674               6            5              61            4   
  4      8              36            5              56            0   
  20   229               5            5              84            0   
  37   904               0            5              15            5   
  25    72               8            4              45            0   
  14   268              11            4               5            0   
  21   234              25            4              17            2   
  36   831               8            3              14            2   
  24   679              12            3              23            0   
  13   121              14            3              11            1   
  27   771              13            3               4            0   
  38  1005               8            3               9            3   
  45   926               4            2               8            2   
  40   222               9            2               3            0   
  49    34               6            2               8            0   
  28  1004               0            2               8            1   
  22   737               2            2              43            2   
  1    220              20            2              72            0   
  26   590               1            1              20            2   
  33   834               0            0               9            1   
  48   959               0            0              13            0   
  46   886               0            0               9            1   
  44   874               0            0               4            1   
  43   761               0            0               5            1   
  15   747               3            0              19            4   
  34   731               1            0               6            1   
  29   803               4            0              11            1   
  39   655               4            0              15            1   
  30   412               0            0               3            0   
  31   920               1            0              72            2   
  35   570               5            0               3            0   
  23   884               2            0              41            7   
  
      post_count  posts_read  days_visited  \
  0          104         951            86   
  2           50         473            77   
  5           25        1062            86   
  18          10         286            59   
  8           16         296            40   
  3           35        1194       

**Forum users, sorted by likes received**

These are folks that posted things other people in the Community Forum liked.

In [18]:
plot_user_data(users.head(50), 'likes_received')

Chart({
  data:       id  likes_received  likes_given  topics_entered  topic_count  \
  2      3             103           86             102            9   
  0      6             101          163             257            7   
  4      8              36            5              56            0   
  8      7              26           33              69            0   
  21   234              25            4              17            2   
  6    116              22           19              89            1   
  1    220              20            2              72            0   
  5     11              16           78             261            0   
  11    99              16           18              73            4   
  18    30              15           61              37            3   
  3     73              14           31             288            3   
  13   121              14            3              11            1   
  27   771              13            3               4            0   
  16    59              13            9              36            0   
  24   679              12            3              23            0   
  14   268              11            4               5            0   
  12   217              10           17              34            1   
  40   222               9            2               3            0   
  38  1005               8            3               9            3   
  36   831               8            3              14            2   
  25    72               8            4              45            0   
  7    198               8           18              85            1   
  32    77               7            8             139            0   
  41    24               7            5             148            2   
  49    34               6            2               8            0   
  17   674               6            5              61            4   
  9     37               6           16               7            1   
  20   229               5            5              84            0   
  35   570               5            0               3            0   
  10   303               4           10             119            3   
  45   926               4            2               8            2   
  29   803               4            0              11            1   
  39   655               4            0              15            1   
  15   747               3            0              19            4   
  47   164               3           10              37            0   
  23   884               2            0              41            7   
  22   737               2            2              43            2   
  19   485               2           28              35            0   
  26   590               1            1              20            2   
  31   920               1            0              72            2   
  34   731               1            0               6            1   
  42   660               1            8               4            0   
  30   412               0            0               3            0   
  48   959               0            0              13            0   
  46   886               0            0               9            1   
  33   834               0            0               9            1   
  44   874               0            0               4            1   
  43   761               0            0               5            1   
  37   904               0            5              15            5   
  28  1004               0            2               8            1   
  
      post_count  posts_read  days_visited  \
  2           50         473            77   
  0          104         951            86   
  4           25         263            89   
  8           16         296            40   
  21           9          83            22   
  6           19         362       

In [19]:
%%html
<script src="https://cdn.rawgit.com/parente/4c3e6936d0d7a46fd071/raw/65b816fb9bdd3c28b4ddf3af602bfd6015486383/code_toggle.js"></script>